# Day 18 - Snailfish

## Read input

In [1]:
from utils import read_input

ABORT = [False]
num_input = read_input(18, eval)

## Part 1

## Let's build up our snailfish calculus

We start by adding `+` operation via `add` function. It takes two lists and returns a list with parameters as its items.

In [2]:
# ADD MATH

def add(a, b):
    return [a, b]

def is_int(n):
    return type(n) == int

In [3]:
# SPLIT MATH
import math

def should_split(number):
    if is_int(number):
        return number >= 10
    
    left, right = number
    return should_split(left) or should_split(right)

def split(number):
    if ABORT[0]:
        return number
    if is_int(number):
        if number >= 10:
            ABORT[0] = True
            return num_split(number)
        else:
            return number
        
    left, right = number
    return [split(left), split(right)]

def num_split(natural_number):
    left = math.floor(natural_number / 2)
    right = math.ceil(natural_number / 2)
    return [left, right]

Explosions all around.

In [14]:
# EXPLODE MATH
from collections import namedtuple

E = namedtuple('E', ["number", "left", "right"])
Extras = namedtuple('EX', ['fromleft', "fromright"])

def is_regular_number_pair(number):
    match number:
        case E() as e:
            number = e.number
    if is_int(number):
        return False
    left, right = number
    return is_int(left) and is_int(right)
    
    
def replace_leftmost(tree, value):
    if is_int(tree[0]):
        tree[0] += value
    else:
        tree[0] = replace_leftmost(tree[0], value)
    return tree

def replace_rightmost(tree, value):
    if is_int(tree[1]):
        tree[1] += value
    else:
        tree[1] = replace_rightmost(tree[1], value)
    return tree

def explode(number, d, extras):
    if is_int(number):
        return number

    if ABORT[0]:
        if extras.fromleft:
            if is_int(number[0]):
                number[0] += extras.fromleft
            else:
                number = replace_leftmost(number, extras.fromleft)
        if False and extras.fromright:
            if is_int(number[1]):
                number[1] += extras.fromright
            else:
                number = replace_rightmost(number, extras.fromright)
        return E(number, left=None, right=None)
    
    # Cause explosion
    if is_regular_number_pair(number) and d >= 4:
        
        ABORT[0] = True
        match number:
            case E() as e:
                return E(0, e.left, e.right)
            case _:
                return E(0, number[0], number[1])
    
    e_left, e_right = None, None
    left = explode(number[0], d+1, extras)
    match left:
        case E() as e:
            number[0] = e.number
            if is_int(number[1]) and e.right is not None:
                number[1] += e.right
            else:
                extras = Extras(fromleft=e.right, fromright=extras.fromright)
            e_left = e.left
            
        case _:
            number[0] = left
    
    right = explode(number[1], d+1, extras)
    match right:
        case E() as e:
            number[1] = e.number
            if is_int(number[0]) and e.left is not None:
                number[0] += e.left
            else:
                extras = Extras(fromleft=extras.fromleft, fromright=e.left)
            e_right = e.right
        case _:
            number[1] = right
    return E(number, e_left, e_right)

def depth(number):
    left, right = number
    nl = is_int(left)
    nr = is_int(right)
    if nl and nr:
        return 1
    
    if nl and not nr:
        return 1 + max(1, depth(right))
    if not nl and nr:
        return 1 + max(depth(left), 1)
    
    return 1+ max(depth(left), depth(right))

def should_explode(number):
    d = depth(number) - 1
    return d >= 4

Reducing

In [5]:
# REDUCER
    
def reduce(number):
    keep_reducing = True
    # If any nested in four pairs, leftmost pair explodes
    # If any number >= 10 leftmost number splits
    # Repeat until no such thing happens
    # Always go back to start after applying any operation
    while keep_reducing:
        if should_explode(number):
            number = explode(number, 0, Extras(None, None)).number
            ABORT[0] = False
            continue
        if should_split(number):
            number = split(number)
            ABORT[0] = False
            continue
        keep_reducing = False

    return number

## Part 1

In [6]:
def main_loop(num_input):
    ABORT[0] = False
    number = num_input[0]
    for new_number in num_input[1:]:
        number = add(number, new_number)
        number = reduce(number)
    return number

ex_1 = [
    [1,1],
    [2,2],
    [3,3],
    [4,4],
]

ex_2 = [
    [1,1],
    [2,2],
    [3,3],
    [4,4],
    [5,5]
]

ex_3 = [
    [1,1],
    [2,2],
    [3,3],
    [4,4],
    [5,5],
    [6,6]
]

main_loop(ex_2)

[[[[3, 0], [5, 3]], [4, 4]], [5, 5]]

In [20]:
import unittest
import inspect


class ExampleTests(unittest.TestCase):


    def test_single_explodes(self):
        print("::TEST::", inspect.stack()[0][3])
        examples = [
            ([[[[[9,8],1],2],3],4], [[[[0,9],2],3],4]),
            ([7,[6,[5,[4,[3,2]]]]], [7,[6,[5,[7,0]]]]),
            ([[6,[5,[4,[3,2]]]],1], [[6,[5,[7,0]]],3]),
            ([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]], [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]),
            ([[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]], [[3,[2,[8,0]]],[9,[5,[7,0]]]])
        ]
        
        for inp, out in examples:
            ABORT[0] = False
            self.assertEqual(explode(inp, 0, Extras(None, None)).number, out)
     

    def test_first_full_example(self):
        print("::TEST::", inspect.stack()[0][3])
        numbers = [
            [[[[4,3],4],4],[7,[[8,4],9]]],
            [1,1]
        ]
        
        add_result = add(*numbers)
        self.assertEqual(add_result, [[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]])
        
        ABORT[0] = False
        first_explode = explode(add_result, 0, Extras(None, None)).number
        fe_out = [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
        self.assertEqual(first_explode, fe_out)
        
        
        ABORT[0] = False
        second_explode = explode(first_explode, 0, Extras(None, None)).number
        se_out = [[[[0,7],4],[15,[0,13]]],[1,1]]
        self.assertEqual(second_explode, se_out)
        
        self.assertFalse(should_explode(se_out))
        self.assertTrue(should_split(second_explode))
        
        ABORT[0] = False
        
        
        first_split = split(second_explode)
        fs_out = [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
        
        self.assertEqual(first_split, fs_out)
        
        self.assertTrue(should_split(second_explode))
        
        ABORT[0] = False
        
        second_split = split(first_split)
        ss_out = [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
        
        self.assertEqual(second_split, ss_out)
    

        res = main_loop(numbers)
        out = [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
        
        self.assertEqual(res, out)
    
    def test_next_examples(self):
        print("::TEST::", inspect.stack()[0][3])
        
        examples = [
            ([[1,1], [2,2], [3,3], [4,4]], [[[[1,1], [2,2]],[3,3]],[4,4]]),
            ([[1,1], [2,2], [3,3], [4,4], [5,5]], [[[[3,0],[5,3]],[4,4]],[5,5]]),
            ([[1,1], [2,2], [3,3], [4,4], [5,5], [6,6]], [[[[5,0],[7,4]],[5,5]],[6,6]]),
        ]
        
        for t_case, (inp, out) in enumerate(examples):
            res = main_loop(inp)
            self.assertEqual(res, out)
            

    def test_slightly_larger_example(self):
        print("::TEST::", inspect.stack()[0][3])
        inp = [
            [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],
            [7,[[[3,7],[4,3]],[[6,3],[8,8]]]],
            #[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]],
            #[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]],
            #[7,[5,[[3,8],[1,4]]]],
            #[[2,[2,2]],[8,[8,1]]],
            #[2,9],
            #[1,[[[9,3],9],[[9,0],[0,7]]]],
            #[[[5,[7,4]],7],1],
            #[[[[4,2],2],6],[8,7]],
        ]
        
        out = [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]] #[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]
        
        ABORT[0] = False
        after_add = add(inp[0], inp[1])
        after_ex1 = explode(after_add,0, Extras(None, None)).number
        ABORT[0] = False
        print(after_ex1)
        after_ex2 = explode(after_ex1,0, Extras(None, None)).number
        print(after_ex2)
        
  #      res = main_loop(inp)
#        print(res)
 #       print(out)
   #     self.assertEqual(res, out)
            
unittest.main(argv=[''], verbosity=0, exit=False)


::TEST:: test_first_full_example
::TEST:: test_next_examples
::TEST:: test_single_explodes
::TEST:: test_slightly_larger_example
[[[[4, 0], [5, 0]], [[[4, 5], [2, 6]], [9, 5]]], [7, [[[3, 7], [4, 3]], [[6, 3], [8, 8]]]]]
[[[[4, 0], [5, 0]], [[0, [7, 6]], [9, 5]]], [7, [[[3, 7], [4, 3]], [[6, 3], [8, 8]]]]]


----------------------------------------------------------------------
Ran 4 tests in 0.040s

OK
